In [1]:
import numpy as np
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from mpl_toolkits.mplot3d import axes3d, Axes3D 
from sklearn.metrics import r2_score
import os
import time
os.environ["PATH"] += "/group/w47/ddalakoti/jupyter-dir/geppy"
import geppy as gep
import operator
#from deap import tools, creator, base 
import random
import sympy as sp
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import RidgeCV, LassoCV, Lasso, SGDRegressor
from scipy.signal import savgol_filter
import matplotlib.ticker as ticker
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
import pickle
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, GaussianNoise
from keras.losses import logcosh
from sklearn.svm import SVR
import glob
#import xgboost as xgb
from sklearn.model_selection import train_test_split
import math

#from numpy import exp


Using TensorFlow backend.


In [11]:
def read_data(loc,d,delU):
    Gij = np.loadtxt(loc+'/Gij_'+d+'.dat')
    Tij = np.loadtxt(loc+'/Tij_'+d+'.dat')
    K   = np.loadtxt(loc+'/k-eps_'+d+'.dat')    
    meanVelGrad = np.loadtxt(loc+'/mean_vel_grad_'+d+'.dat')
    Lij = np.loadtxt(loc+'/lambda_'+d+'.dat')
    Reyn = np.loadtxt(loc+'/Reyn_'+d+'.dat')
    rho = np.loadtxt(loc+'/rho_'+d+'.dat')
    mv = np.loadtxt(loc+'/mean_vel_'+d+'.dat')
    
    #mvgg = np.loadtxt(loc+'/mvgg_'+d+'.dat')
    
    idel = [3, 6, 7, 12, 15, 16, 21, 24, 25]
    
    #mvgg = np.delete(mvgg,idel,axis=1)
    delta = np.ones(Reyn.shape[0])*np.sum(rho*(delU/2-mv[:,0])*(delU/2+mv[:,0]))*7.34e-6/(0.68*delU**2)
    #delta = np.ones(Reyn.shape[0])*100/np.max(meanVelGrad[:,1])
    x = np.linspace(0,0.0423,576)
    x = x-0.0423/2.0
    eps = np.copy(K[:,1])
    K   = K[:,0]
    index = np.where(K>0.05*np.max(K))
    t_time = np.zeros(576)
    t_time[:] = int(d)*3.38e-7
    
    loend = index[0][0]
    hiend = index[0][-1]
    #loend = 100
    #hiend = 476
    #Gij = Gij[loend:hiend,:]
    #Tij = Tij[loend:hiend,:]
    #meanVelGrad = meanVelGrad[loend:hiend,:]
    #eps = eps[loend:hiend]
    #K = K[loend:hiend] 
    #Lij = Lij[loend:hiend]
    #Reyn = Reyn[loend:hiend]
    #delta = delta[loend:hiend]
    #x = x[loend:hiend]/delta
    #t_time = t_time[loend:hiend]
        
    
    return Gij, Tij, meanVelGrad, eps, K, Lij, Reyn, delta,x,t_time

    

    

In [ ]:
def get_basis_vect(b,Sij,omega,meanVelGrad,Reyn,Tij,eps,K,Lij,x,t_time,mvgg):
        # basis for CNN just, Tij and meanVelGrad\n",
        basis_vect = np.zeros((b.shape[0],3,3,51))
        C = np.transpose(meanVelGrad,(0,2,1))

        basis_vect[:,:,:,0] = b
        basis_vect[:,:,:,1] = meanVelGrad
        basis_vect[:,:,:,2] = C
        basis_vect[:,:,:,3] = Lij
        basis_vect[:,:,:,4] = np.matmul(meanVelGrad,Lij)
        basis_vect[:,:,:,5] = np.matmul(Lij,meanVelGrad)
        
        basis_vect[:,:,:,6] = np.matmul(C,Lij)
        basis_vect[:,:,:,7] = np.matmul(Lij,C)
        
        basis_vect[:,:,:,8] = np.matmul(C,b)
        basis_vect[:,:,:,9] = np.matmul(b,C)
        basis_vect[:,:,:,10] = np.matmul(b,Lij)
        
        
        
        basis_vect[:,:,:,11] = np.matmul(meanVelGrad,b)
        basis_vect[:,:,:,12] = np.matmul(b,meanVelGrad)
        
        
        
        basis_vect[:,:,:,13] = K[:,None,None]
        basis_vect[:,:,:,14] = eps[:,None,None]
        basis_vect[:,:,:,15] = Reyn[:,None,None]
        basis_vect[:,:,:,16] = np.trace(np.matmul(b,b),axis1=1,axis2=2)[:,None,None]
        
        basis_vect[:,:,:,17] = np.trace(C,axis1=1,axis2=2)[:,None,None]
        basis_vect[:,:,:,18] = np.trace(np.matmul(C,C),axis1=1,axis2=2)[:,None,None]
        basis_vect[:,:,:,19] = np.trace(Lij,axis1=1,axis2=2)[:,None,None]
        
        basis_vect[:,:,:,20] = np.trace(np.matmul(meanVelGrad,b),axis1=1,axis2=2)[:,None,None]
    
        basis_vect[:,:,:,21] = np.trace(np.matmul(b,Lij),axis1=1,axis2=2)[:,None,None]
        basis_vect[:,:,:,22] = np.trace(np.matmul(meanVelGrad,Lij),axis1=1,axis2=2)[:,None,None]
        
        
        
        
        return basis_vect       


In [12]:
def get_training_data_time(dir_loc,time,normalise=False,normalise_const=True,delU=100):

    files = glob.glob(dir_loc+'accu*.dat')
    indexes = [F[-7:-4] for F in files]        
    loindices = np.zeros(len(indexes)+1)
    Gij, Tij , meanVelGrad , eps, K, Lij, Reyn, delta, x, t_time = read_data(dir_loc,indexes[time],delU)
    loindices[1] = K.shape[0]
    
    b = np.copy(Tij)
    b[:,0] = b[:,0]-2.0*K/3.0
    b[:,4] = b[:,4]-2.0*K/3.0
    b[:,8] = b[:,8]-2.0*K/3.0
    Gij = np.reshape(Gij,(Gij.shape[0],3,3))
    Tij = np.reshape(Tij,(Tij.shape[0],3,3))
    meanVelGrad = np.reshape(meanVelGrad,(meanVelGrad.shape[0],3,3))
    b = np.reshape(b,(b.shape[0],3,3))
    Lij = np.reshape(Lij,(Lij.shape[0],3,3))

    Sij = np.zeros((b.shape[0],3,3))
    omega = np.zeros((b.shape[0],3,3))
    
    for i in range(Sij.shape[0]):
        Sij[i,:,:] = 0.5*(meanVelGrad[i,:,:]+np.transpose(meanVelGrad[i,:,:]))
        omega[i,:,:] = 0.5*(meanVelGrad[i,:,:]-np.transpose(meanVelGrad[i,:,:]))
        
    b = b/(2.0*K[:,None,None])
    if(normalise):    
        Gij = Gij*K[:,None,None]/eps[:,None,None]
        #Gij = Gij/(eps[:,None,None])
        
        Tij = Tij/K[:,None,None]
        meanVelGrad = meanVelGrad*K[:,None,None]/eps[:,None,None]
        Sij = Sij*K[:,None,None]/eps[:,None,None]
        omega = omega*K[:,None,None]/eps[:,None,None]
        Lij = Lij*K[:,None,None]
    
    if(normalise_const):
        
        Gij = Gij*delta[:,None,None]/delU
        #Gij = Gij*delta[:,None,None]/delU**3        
        Tij = Tij/delU**2
        meanVelGrad = meanVelGrad*delta[:,None,None]/delU
        Sij = Sij*delta[:,None,None]/delU
        omega = omega*delta[:,None,None]/delU
        eps = eps*delta/delU**3
        K = K/delU**2
        Lij = Lij*delU**2
        t_time = t_time*delU/delta
  
    #basis_vect = get_basis_vect(b,Sij,omega,meanVelGrad,Reyn,Tij,eps,K,Lij,x,t_time,mvgg)
    #basis_vect = np.moveaxis(basis_vect,0,-2)
    #Gij = np.moveaxis(Gij,0,-1)
    #Tij = np.moveaxis(Tij,0,-1)
    #Lij = np.moveaxis(Lij,0,-1)
    #b = np.moveaxis(b,0,-1)
    #meanVelGrad = np.moveaxis(meanVelGrad,0,-1)
    #Sij = np.moveaxis(Sij,0,-1)
    #omega = np.moveaxis(omega,0,-1)
        
    return Gij, Tij, meanVelGrad, b, Sij, omega, Lij, eps, K, Reyn


In [13]:
def get_training(direc,ist,nt,stp, delU,nfs,i1,j1):
            Gij, Tij, meanVelGrad, b, Sij, omega, Lij, eps, K, Reyn = get_training_data_time(direc,ist,False,False,delU)
            print(Gij.shape, Tij.shape)
            for i in range(ist+stp,ist+nt,stp):
                Gij1, Tij1, meanVelGrad1, b1, Sij1, omega1, Lij1, eps1, K1, Reyn1 = get_training_data_time(direc,i,False,False,delU)
                Gij = np.append(Gij,Gij1,axis=0)
                Tij = np.append(Tij,Tij1,axis=0)
                meanVelGrad = np.append(meanVelGrad,meanVelGrad1,axis=0)
                b = np.append(b,b1,axis=0)
                Sij = np.append(Sij,Sij1,axis=0)
                omega = np.append(omega,omega1,axis=0)
                Lij = np.append(Lij,Lij1,axis=0)
                eps = np.append(eps,eps1,axis=0)
                K = np.append(K,K1,axis=0)
                Reyn = np.append(Reyn,Reyn1,axis=0)
                
           
            return Gij, Tij, meanVelGrad, b, Sij, omega, Lij, eps, K , Reyn

In [14]:
def scale_data(X_train,X_test, X_val, Y_train, Y_test, Y_val, which):
    
    if(which == 'std'):
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        X_val = sc.transform(X_val)
        Y_train = sc.fit_transform(Y_train.reshape(-1,1))
        Y_test = sc.transform(Y_test.reshape(-1,1))
        Y_val = sc.transform(Y_val.reshape(-1,1))
        return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which == 'std2'):
        sc = StandardScaler(with_mean=False)
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        X_val = sc.transform(X_val)
        Y_train = sc.fit_transform(Y_train.reshape(-1,1))
        Y_test = sc.transform(Y_test.reshape(-1,1))
        Y_val = sc.transform(Y_val.reshape(-1,1))
        return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which == 'mean'):
        meanX   = np.mean(X_train,axis=0)
        stdX    = np.std(X_train,axis=0)
    
        meanY   = np.mean(Y_train)
        stdY    = np.std(Y_train)
    
        Y_train = (Y_train)/meanY
        Y_test = (Y_test)/meanY
        Y_val   = (Y_val)/meanY
        
        for i in range(X_train.shape[1]):
            X_train[:,i] = (X_train[:,i])/meanX[i]
            X_test[:,i] = (X_test[:,i])/meanX[i]
            X_val[:,i] = (X_val[:,i])/meanX[i]
        return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    
    if(which=='minmax'):
        minX   = np.min(X_train,axis=(0))
        maxX    = np.max(X_train,axis=(0))
    
        minY   = np.min(Y_train)
        maxY    = np.max(Y_train)
    
        Y_train = (Y_train-minY)/(maxY-minY)
        Y_test = (Y_test-minY)/(maxY-minY)
        Y_val   = (Y_val-minY)/(maxY-minY)
        #Y_train = (Y_train)/(maxY-minY)
        #Y_test = (Y_test)/(maxY-minY)
        #Y_val   = (Y_val)/(maxY-minY)
    
        for i in range(X_train.shape[1]):
            X_train[:,i] = (X_train[:,i]-minX[i])/(maxX[i]-minX[i])
            X_test[:,i] = (X_test[:,i]-minX[i])/(maxX[i]-minX[i])            
            X_val[:,i] = (X_val[:,i]-minX[i])/(maxX[i]-minX[i])
            
        return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which=='max'):
        maxX    = np.max(np.abs(X_train),axis=(0))
    
        maxY    = np.max(np.abs(Y_train))
    
        Y_train = (Y_train)/(maxY)
        Y_test = (Y_test)/(maxY)
        Y_val   = (Y_val)/(maxY)
        
    
        for i in range(X_train.shape[1]):
            X_train[:,i] = (X_train[:,i])/(maxX[i])
            X_test[:,i] = (X_test[:,i])/(maxX[i])            
            X_val[:,i] = (X_val[:,i])/(maxX[i])
            
        return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which == 'yeo'):
            pt = PowerTransformer(standardize=True)
            X_train = pt.fit_transform(X_train)            
            X_test  = pt.transform(X_test)
            X_val   = pt.transform(X_val)
            Y_train  = pt.fit_transform(Y_train.reshape(-1,1))            
            Y_test  = pt.transform(Y_test.reshape(-1,1))
            Y_val   = pt.transform(Y_val.reshape(-1,1))
            return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which == 'quant'):
                pt = QuantileTransformer(output_distribution='normal')
                X_train = pt.fit_transform(X_train)
                X_test  = pt.transform(X_test)
                X_val   = pt.transform(X_val)
                Y_train = pt.fit_transform(Y_train.reshape(-1,1))
                Y_test  = pt.transform(Y_test.reshape(-1,1))
                Y_val   = pt.transform(Y_val.reshape(-1,1))
                return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which=='log'):
               X_train = do_log(X_train)    
               X_test = do_log(X_test)    
               X_val = do_log(X_val)
               Y_train = do_log2(Y_train)
               Y_test = do_log2(Y_test)
               Y_val = do_log2(Y_val)
               X_train, X_test, X_val, Y_train, Y_test, Y_val = scale_data(X_train, X_test,  X_val, Y_train, Y_test,  Y_val,'std')

               return X_train, X_test, X_val, Y_train, Y_test, Y_val
    if(which=='exp'):
               X_train = X_train**0.3    
               X_test = X_test**0.3    
               X_val = X_val**0.3
               Y_train = Y_train**0.3
               Y_test = Y_test**0.3
               Y_val = Y_val**0.3
               X_train, X_test, X_val, Y_train, Y_test, Y_val = scale_data(X_train, X_test,  X_val, Y_train, Y_test,  Y_val,'std')

               return X_train, X_test, X_val, Y_train, Y_test, Y_val
    if(which=='median'):
        RS = RobustScaler()
        X_train = RS.fit_transform(X_train)
        X_test = RS.transform(X_test)
        X_val = RS.transform(X_val)
        Y_train = RS.fit_transform(Y_train.reshape(-1,1))
        Y_test = RS.transform(Y_test.reshape(-1,1))
        Y_val = RS.transform(Y_val.reshape(-1,1))
        return X_train, X_test, X_val, Y_train, Y_test, Y_val


        

    

In [15]:
def scale_data_all(X, which):
    
    if(which == 'std'):
        sc = StandardScaler()
        for i in range(0,3):
            for j in range(0,3):
                X[:,i,j] = sc.fit_transform(X[:,i,j].reshape(-1,1))[:,0]
                
        return X
    
    if(which == 'std2'):
        sc = StandardScaler(with_mean=False)
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        X_val = sc.transform(X_val)
        Y_train = sc.fit_transform(Y_train.reshape(-1,1))
        Y_test = sc.transform(Y_test.reshape(-1,1))
        Y_val = sc.transform(Y_val.reshape(-1,1))
        return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which == 'mean'):
        meanX   = np.mean(X_train,axis=0)
        stdX    = np.std(X_train,axis=0)
    
        meanY   = np.mean(Y_train)
        stdY    = np.std(Y_train)
    
        Y_train = (Y_train)/meanY
        Y_test = (Y_test)/meanY
        Y_val   = (Y_val)/meanY
        
        for i in range(X_train.shape[1]):
            X_train[:,i] = (X_train[:,i])/meanX[i]
            X_test[:,i] = (X_test[:,i])/meanX[i]
            X_val[:,i] = (X_val[:,i])/meanX[i]
        return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    
    if(which=='minmax'):
        minX   = np.min(X_train,axis=(0))
        maxX    = np.max(X_train,axis=(0))
    
        minY   = np.min(Y_train)
        maxY    = np.max(Y_train)
    
        Y_train = (Y_train-minY)/(maxY-minY)
        Y_test = (Y_test-minY)/(maxY-minY)
        Y_val   = (Y_val-minY)/(maxY-minY)
        #Y_train = (Y_train)/(maxY-minY)
        #Y_test = (Y_test)/(maxY-minY)
        #Y_val   = (Y_val)/(maxY-minY)
    
        for i in range(X_train.shape[1]):
            X_train[:,i] = (X_train[:,i]-minX[i])/(maxX[i]-minX[i])
            X_test[:,i] = (X_test[:,i]-minX[i])/(maxX[i]-minX[i])            
            X_val[:,i] = (X_val[:,i]-minX[i])/(maxX[i]-minX[i])
            
        return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which=='max'):
        maxX    = np.max(np.abs(X_train),axis=(0))
    
        maxY    = np.max(np.abs(Y_train))
    
        Y_train = (Y_train)/(maxY)
        Y_test = (Y_test)/(maxY)
        Y_val   = (Y_val)/(maxY)
        
    
        for i in range(X_train.shape[1]):
            X_train[:,i] = (X_train[:,i])/(maxX[i])
            X_test[:,i] = (X_test[:,i])/(maxX[i])            
            X_val[:,i] = (X_val[:,i])/(maxX[i])
            
        return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which == 'yeo'):
            pt = PowerTransformer(standardize=True)
            X_train = pt.fit_transform(X_train)            
            X_test  = pt.transform(X_test)
            X_val   = pt.transform(X_val)
            Y_train  = pt.fit_transform(Y_train.reshape(-1,1))            
            Y_test  = pt.transform(Y_test.reshape(-1,1))
            Y_val   = pt.transform(Y_val.reshape(-1,1))
            return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which == 'quant'):
                pt = QuantileTransformer(output_distribution='normal')
                X_train = pt.fit_transform(X_train)
                X_test  = pt.transform(X_test)
                X_val   = pt.transform(X_val)
                Y_train = pt.fit_transform(Y_train.reshape(-1,1))
                Y_test  = pt.transform(Y_test.reshape(-1,1))
                Y_val   = pt.transform(Y_val.reshape(-1,1))
                return X_train, X_test, X_val, Y_train, Y_test, Y_val
    
    if(which=='log'):
               X_train = do_log(X_train)    
               X_test = do_log(X_test)    
               X_val = do_log(X_val)
               Y_train = do_log2(Y_train)
               Y_test = do_log2(Y_test)
               Y_val = do_log2(Y_val)
               X_train, X_test, X_val, Y_train, Y_test, Y_val = scale_data(X_train, X_test,  X_val, Y_train, Y_test,  Y_val,'std')

               return X_train, X_test, X_val, Y_train, Y_test, Y_val
    if(which=='exp'):
               X_train = X_train**0.3    
               X_test = X_test**0.3    
               X_val = X_val**0.3
               Y_train = Y_train**0.3
               Y_test = Y_test**0.3
               Y_val = Y_val**0.3
               X_train, X_test, X_val, Y_train, Y_test, Y_val = scale_data(X_train, X_test,  X_val, Y_train, Y_test,  Y_val,'std')

               return X_train, X_test, X_val, Y_train, Y_test, Y_val
    if(which=='median'):
        RS = RobustScaler()
        X_train = RS.fit_transform(X_train)
        X_test = RS.transform(X_test)
        X_val = RS.transform(X_val)
        Y_train = RS.fit_transform(Y_train.reshape(-1,1))
        Y_test = RS.transform(Y_test.reshape(-1,1))
        Y_val = RS.transform(Y_val.reshape(-1,1))
        return X_train, X_test, X_val, Y_train, Y_test, Y_val


        

    

In [16]:
def do_inverse_scaling(Y_sc, Y_un, which):
    if(which=='std'):
        meanY = np.mean(Y_un)
        stdY = np.std(Y_un)
        return Y_sc*stdY + meanY
    if(which=='minmax'):
        minY = np.min(Y_un)
        maxY = np.max(Y_un)
        return Y_sc*(maxY-minY)+minY
    if(which=='max'):
        maxY = np.max(np.abs(Y_un))
        return Y_sc*maxY
    
    if(which=='median'):
        RS = RobustScaler()
        RS.fit(Y_un.reshape(-1,1))
        return RS.inverse_transform(Y_sc.reshape(-1,1))
    if(which == 'yeo'):
        pt = PowerTransformer(standardize=True)
        Y_train  = pt.fit(Y_un.reshape(-1,1))            
        return pt.inverse_transform(Y_sc.reshape(-1,1))
    return Y_sc

In [ ]:
npts=576
np.random.seed(10)
nT = [i for i in range(0,100)]
np.random.shuffle(nT)


In [17]:
def get_data(i1,j1,nfs):
    #nfs = 19
    #X, Y, _ = get_training('./data/CaseF_scaled/',0,100,2,80,nfs,i1,j1)
    #X1, Y1, _ = get_training('./data/CaseC_scaled/',50,150,3,60,nfs,i1,j1)
    X, Y, _ = get_training('./data/new_data/40_scaled/',0,50,1,80,nfs,i1,j1)
    X1, Y1, _ = get_training('./data/new_data/60_scaled/',0,50,1,60,nfs,i1,j1)
    
    
    #X, Y, _ = get_CNN_training('./data/50_new_scaled/',0,100,2,100,nfs,i1,j1)
    
    X = np.append(X,X1,axis=0)
    Y = np.append(Y,Y1,axis=0)
    #nfs=19
    npts=576
    X_train = np.zeros((npts*80,nfs))
    Y_train = np.zeros((npts*80,1))

    X_val = np.zeros((npts*20,nfs))
    Y_val = np.zeros((npts*20,1))

    for i in range(0,80):
        X_train[i*npts:(i+1)*npts,:] = X[nT[i]*npts:(nT[i]+1)*npts,:]
        Y_train[i*npts:(i+1)*npts] = Y[nT[i]*npts:(nT[i]+1)*npts,None]
    j=0
    for i in range(80,100):
        X_val[j*npts:(j+1)*npts,:] = X[nT[i]*npts:(nT[i]+1)*npts,:]
        Y_val[j*npts:(j+1)*npts] = Y[nT[i]*npts:(nT[i]+1)*npts,None]
        j=j+1
        
    #nfs=19
    minY = np.min(Y_train)
    maxY = np.max(Y_train)
    stdY = np.std(Y_train)
    meanY = np.mean(Y_train)
    Y_un = np.copy(Y_train)

    X_test, Y_test, _ = get_training('./data/new_data/75_scaled/',0,60,6,100,nfs,i1,j1)
    #X_test, Y_test, _ = get_CNN_training('./data/CaseE_scaled/',0,80,2,150,nfs,i1,j1)
    #X_test, Y_test, _ = get_CNN_training('./data/CaseE_scaled/',0,60,3,150,nfs,i1,j1)
    
    
    X_train, X_test, X_val, Y_train, Y_test, Y_val = scale_data(X_train, X_test,  X_val, Y_train, Y_test,  Y_val,'std')
    return X_train, X_test, X_val, Y_train, Y_test, Y_val, Y_un


In [19]:
Gij, Tij, meanVelGrad, b, Sij, omega, Lij, eps, K , Reyn = get_training('./data/60_scaled/',0,50,50,60,23,0,0)
#Gij2, Tij2, meanVelGrad2, b2, Sij2, omega2, Lij2, eps2, K2 , Reyn2 = get_training('./data/new_data/60_scaled/',0,50,10,60,23,0,0)


(576, 3, 3) (576, 3, 3)


In [ ]:
eps = eps[:,None,None]
K = K[:,None,None]
Reyn = Reyn[:,None,None]


In [ ]:
Gij = scale_data_all(Gij,'std')
Lij = scale_data_all(Lij,'std')
b = scale_data_all(b,'std')
meanVelGrad = scale_data_all(meanVelGrad,'std')


In [ ]:
def protected_div(x1, x2):
    if abs(x2) < 1e-6:
        return 1
    return x1 / x2

def protected_exp(x1):
    if abs(x1) > 2:
        return 1
    return np.exp(x1)
def protected_log(x1):
    if x1 <= 1:
        return 1
    return np.log(x1)
exp = np.vectorize(protected_exp)
log = np.vectorize(protected_log)
def mult_three(*args):    
    out=1
    for i in range(len(args)):
        out = out*args[i]      
    return out

def add_three(*args):    
    out=0
    for i in range(len(args)):
        out = out+args[i]      
    return out

def matmul(x1,x2):    
    if(np.isscalar(x1) or np.isscalar(x2)):
        return x2*x2
    if(x1.shape[1]==1 or x2.shape[1]==1):
        return x1*x2
    
    
    return np.matmul(x1,x2)

def transpose(x1):
    
    if(np.isscalar(x1)):
        return x1
        
    return np.transpose(x1)
    

def trace(x1):
    
    if(np.isscalar(x1)):
        return x1
    return np.trace(x1)


In [ ]:
pset = gep.PrimitiveSet('main', input_names=['B1','B2','B3'])
#pset.add_function(operator.add, 2)
#pset.add_function(operator.sub, 2)
pset.add_function(matmul, 2)
pset.add_function(transpose,1)

pset.add_rnc_terminal()
#pset.add_plasmid_terminal()

pset2 = gep.PrimitiveSet('main2', input_names=['T1','T2','T3','eps','K','Reyn'])
pset2.add_rnc_terminal()
pset2.add_function(matmul, 2)
pset2.add_function(transpose, 1)
pset2.add_function(trace, 1)


 


In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1,))
creator.create('Individual', gep.Chromosome, fitness=creator.FitnessMin)

h = 8 # head length
n_genes = 6
toolbox = gep.Toolbox()
r=8
toolbox.register('rnc_gen', random.uniform, a=-10, b=10)   # each RNC is random integer within [-5, 5]
toolbox.register('gene_gen', gep.GenePlasmid, pset=pset, head_length=h,rnc_gen=toolbox.rnc_gen, rnc_array_length=r,pset_plasmid=pset2,head_plasmid=5,n_plasmid=1)
#toolbox.register('gene_gen', gep.GeneDc, pset=pset, head_length=h,rnc_gen=toolbox.rnc_gen, rnc_array_length=r)
#toolbox.register('plasmid',  gep.Gene, pset=pset, head_length=h)
toolbox.register('individual', creator.Individual, gene_gen=toolbox.gene_gen, n_genes=n_genes, linker=add_three)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)



In [ ]:
# compile utility: which translates an individual into an executable function (Lambda)
toolbox.register('compile', gep.compile_plasmids, pset=pset, pset_plasmid=pset2)
#toolbox.register('compile', gep.compile_, pset=pset)

In [ ]:
def evaluate(individual):
    """Evalute the fitness of an individual: MSE (mean squared error)"""
    func = toolbox.compile(individual)    
    Yp = np.array(list(map(func, Tij,meanVelGrad)))     
    #print(Yp.shape)
    if(Yp.shape!=Gij.shape):
        return (1e4,) # Some high value
    return abs(np.mean((Gij - Yp) ** 2)),

In [ ]:
func = toolbox.compile(pop[0])    


In [ ]:
def evaluate(individual):
    """Evalute the fitness of an individual: MSE (mean squared error)"""
    func = toolbox.compile(individual)    
    
    Yp = np.array(list(map(func, b,meanVelGrad,Lij,b,meanVelGrad,Lij,eps,K,Reyn)))  
    #print(Yp.shape)
    if(Yp.ndim==1):
        Yp = Yp[:,None,None]
    return np.mean((Gij - Yp)**2),

In [ ]:
Yp = np.array(list(map(func, b,meanVelGrad,Lij,b,meanVelGrad,Lij)))  


In [ ]:
np.mean((Gij - Yp)**2)

In [ ]:
#maxs = np.max(abs(Gij),2)
def evaluate_test(individual):
    """Evalute the fitness of an individual: MSE (mean squared error)"""
    func = toolbox.compile(individual)    
    Yp = func(B1,B2,B3,B4,B5,B6,I,T1,T2,T3,T4,T5,T6,I2,detT)
    #Yp = func(B1,B2,B3,I,T1,T2,T3)
    #Yp = np.array(list(map(func, B1,B2,B3,T1,T2,T3)))     
    #print(Yp.shape)
    #return np.mean(abs(Gij - Yp)), 
    #try:
    #    pr = Yp[1,1,:]
    #except:
    #    pr = Yp
    pr = Yp*eps[None,None,:]/K[None,None,:]       
    
    return np.mean((Gij*eps[None,None,:]/K[None,None,:] - pr)**2),  
 

In [ ]:
def evaluate(individual):
    """Evalute the fitness of an individual: MSE (mean squared error)"""
    func = toolbox.compile(individual)    
    Yp = np.array(list(map(func, b[:,0]/K,H1[:,0]*tau,
                                 H2[:,0]*tau,H3[:,0]*tau,H4[:,0]*tau,
                                 H5[:,0]*tau,H6[:,0]*tau,H7[:,0]*tau,
                                 H8[:,0]*tau,H9[:,0]*tau)))     
    #print(Yp.shape)
    #if(Yp.shape!=Gij.shape):
    #    return (1e4,) # Some high value
    return abs(np.mean((Gij[:,0]*tau - Yp) ** 2)),

In [ ]:
def evaluate_linear_scaling(individual):
    """Evaluate the fitness of an individual with linearly scaled MSE.
    Get a and b by minimizing (a*Yp + b - Y)"""
    func = toolbox.compile(individual)
    Yp = np.array(list(map(func, b[:,0]/K,H1[:,0]*tau,
                                 H2[:,0]*tau,H3[:,0]*tau,H4[:,0]*tau,
                                 H5[:,0]*tau,H6[:,0]*tau,H7[:,0]*tau,
                                 H8[:,0]*tau,H9[:,0]*tau)))     
    Vreal = Gij[:,0]*tau
    
    if isinstance(Yp, np.ndarray):
        Q = np.hstack((np.reshape(Yp, (-1, 1)), np.ones((len(Yp), 1))))
        (individual.a, individual.b), residuals, _, _ = np.linalg.lstsq(Q, Vreal, rcond=None)   
        # residuals is the sum of squared errors
        if residuals.size > 0:
            #return residuals[0] / len(Vreal),   # MSE
            return np.log(1+residuals[0] / len(Vreal)),   # MSE
    
    # for the above special cases, the optimal linear scaling is just the mean of true target values
    individual.a = 0
    individual.b = np.mean(Vreal)
    #return np.mean((Vreal - individual.b) ** 2),
    return np.log(1+np.mean((Vreal - individual.b) ** 2)),

In [ ]:
def evaluate_linear_scaling_tensor(individual):
    """Evaluate the fitness of an individual with linearly scaled MSE.
    Get a and b by minimizing (a*Yp + b - Y)"""
    func = toolbox.compile(individual)
    predict = np.array(list(map(func, b,meanVelGrad,Lij,b,meanVelGrad,Lij,eps,K,Reyn)))  

    #print(predict.shape)
    #Vreal = out
    resid = 0
    slp = np.zeros((3,3))
    intp = np.zeros((3,3))
    for i in range(0,3):
        for j in range(0,3):            
                #print(predict.ndim,i,j)
                try:         
                    Yp = predict[:,i,j]
                except:
                    #print(predict.ndim,i,j,predict.shape)
                    Yp = predict[:,None]
                Q = np.hstack((np.reshape(Yp, (-1, 1)), np.ones((len(Yp), 1))))
                (slp[i,j], intp[i,j]), residuals, _, _ = np.linalg.lstsq(Q, Gij[:,i,j], rcond=None)   
                individual.a = slp
                individual.b = intp
                
                pr = Yp*slp[i,j]+intp[i,j]
                resid = resid + np.mean((abs(Gij[:,i,j]-pr)))
    
        
    return resid,
    

In [ ]:
out = Gij[0,0,:]*eps/K
def evaluate_linear_scaling(individual):
    """Evaluate the fitness of an individual with linearly scaled MSE.
    Get a and b by minimizing (a*Yp + b - Y)"""
    func = toolbox.compile(individual)
    Yp = np.array(list(map(func, S1,S2,S3,S4,S5,S6,S7,S8,
                            T1,T2,T3,T4,T5,T6,T7,T8)))     
    Vreal = out
    Yp = Yp*eps/K
    
    if isinstance(Yp, np.ndarray):
        Q = np.hstack((np.reshape(Yp, (-1, 1)), np.ones((len(Yp), 1))))
        (individual.a, individual.b), residuals, _, _ = np.linalg.lstsq(Q, Vreal, rcond=None)   
        # residuals is the sum of squared errors
        if residuals.size > 0:
            return residuals[0] / len(Vreal),   # MSE
    
    # for the above special cases, the optimal linear scaling is just the mean of true target values
    individual.a = 0
    individual.b = np.mean(Vreal)
    return np.mean((Vreal - individual.b) ** 2),

In [10]:
toolbox.register('evaluate', evaluate)
#toolbox.register('evaluate', evaluate_test)
toolbox.register('select', tools.selTournament, tournsize=10)


NameError: name 'toolbox' is not defined

In [ ]:
# 1. general operators
toolbox.register('mut_uniform', gep.mutate_uniform, pset=pset, ind_pb=0.1,pb=0.8)
toolbox.register('mut_invert', gep.invert, pb=0.1)
toolbox.register('mut_is_transpose', gep.is_transpose, pb=0.1)
toolbox.register('mut_ris_transpose', gep.ris_transpose, pb=0.1)
#toolbox.register('mut_gene_transpose', gep.gene_transpose, pb=0.1)
toolbox.register('cx_1p', gep.crossover_one_point, pb=0.1)
#toolbox.register('cx_2p', gep.crossover_two_point, pb=0.3)
#toolbox.register('cx_gene', gep.crossover_gene, pb=0.1)

# 2. Dc-specific operators
toolbox.register('mut_dc', gep.mutate_uniform_dc, ind_pb=0.1, pb=0.5)
toolbox.register('mut_invert_dc', gep.invert_dc, pb=0.1)

# This doesn't work with plasmid right now
#toolbox.register('mut_transpose_dc', gep.transpose_dc, pb=0.1)

# for some uniform mutations, we can also assign the ind_pb a string to indicate our expected number of point mutations in an individual
toolbox.register('mut_rnc_array_dc', gep.mutate_rnc_array_dc, rnc_gen=toolbox.rnc_gen, ind_pb='0.1p')
toolbox.pbs['mut_rnc_array_dc'] = 0.5  # we can also give the probability via the pbs property

#Plasmid mutation
toolbox.register('mut_unif_plasmid', gep.mutate_uniform_plasmid, pset_plasmid=pset2, ind_pb=0.6,pb=0.5)
toolbox.register('mut_move_plasmid', gep.mutate_move_plasmid, ind_pb=0.2,pb=0.2)
toolbox.register('mut_rnc_array_plasmid', gep.mutate_rnc_array_plasmid,  rnc_gen=toolbox.rnc_gen,ind_pb=0.4,pb=0.5)
#toolbox.register('cx_plasmid', gep.crossover_one_point_plasmid, pb=0.4)


In [ ]:
stats = tools.Statistics(key=lambda ind: ind.fitness.values[0])
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

In [ ]:
# size of population and number of generations
bests = []
loss = []
n_pop = 200
n_gen = 100
champs = 3
for i in range(0,1):
    pop = toolbox.population(n=n_pop) # 
    hof = tools.HallOfFame(champs)   # only record the best three individuals ever found in all generations
    pop, log = gep.gep_simple(pop, toolbox, n_generations=n_gen, n_elites=1,
                           stats=stats,hall_of_fame=hof, verbose=True)
    bests.append(hof[0])
    loss.append(log[n_gen-1]['min'])
    print(i)


In [ ]:
pop, log = gep.gep_simple(pop, toolbox, n_generations=n_gen, n_elites=1,
                           stats=stats,hall_of_fame=hof, verbose=True)

In [ ]:
hof[0][5]

In [ ]:
plt.figure()
plt.plot(avgs)

In [ ]:
avgs  = np.zeros(n_gen)
best  = np.zeros(n_gen)
for i in range(0,n_gen):
    avgs[i] = log[i]['avg']
    best[i] = log[i]['min']

In [ ]:
plt.figure()
plt.plot(avgs)
#plt.plot(best)

In [ ]:
DEFAULT_SYMBOLIC_FUNCTION_MAP = {
    operator.and_.__name__: sp.And,
    operator.or_.__name__: sp.Or,
    operator.not_.__name__: sp.Not,
    operator.add.__name__: operator.add,
    operator.sub.__name__: operator.sub,
    operator.mul.__name__: operator.mul,
    operator.neg.__name__: operator.neg,
    operator.floordiv.__name__: operator.floordiv,
    operator.truediv.__name__: operator.truediv,
    'protected_div': operator.truediv,
    'protected_exp': sp.functions.elementary.exponential.exp,
    'protected_log': sp.functions.elementary.exponential.log,
    'protected_log': sp.functions.elementary.exponential.log,
    'trace' : sp.expressions.Trace,
    'transpose' : sp.expressions.Transpose,
    'matmul' : sp.expressions.MatMul,
    
    
    #math.log.__name__: sp.log
    
}

In [ ]:
for i in range(1):
    ind = hof[i]
    func = gep.simplify(ind)    
    func = ind.a * func + ind.b
    print('Symplified best individual {}: '.format(i))
    print(func)

In [ ]:
f = gep.compile_plasmids(hof[0],pset,pset2)
#f = gep.compile_(hof[0],pset)
#pred = f(B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,I,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10)
predict = np.array(list(map(f, b,meanVelGrad,Lij,b,meanVelGrad,Lij,eps,K,Reyn)))  

#pred = f(b,meanVelGrad,Lij,b,meanVelGrad,Lij)
#pred = f(B1,B2,B3,B4,I)
#pred = f(S1,S2,S3,S4,S5,S6,T1,T2,T3,T4,T5,T6)
#pred = f(S1,S2,S3,S4,S5,S6,S7,S8,T1,T2,T3,T4,T5,T6,T7,T8)
#pred = f(b[:,0]/K,H1[:,0]*tau,H2[:,0]*tau,H3[:,0]*tau,H4[:,0]*tau,H5[:,0]*tau,H6[:,0]*tau,H7[:,0]*tau,H8[:,0]*tau,H9[:,0]*tau)
#red = f(S1,S2,S3,S4,T1,T2,T3,T4)

In [ ]:
gep.simplify(hof[0],DEFAULT_SYMBOLIC_FUNCTION_MAP)

In [ ]:
test = np.trace(Lij,axis1=1,axis2=2)[:,None,None]*(3.0*np.matmul(b,meanVelGrad)) + np.trace(Lij,axis1=1,axis2=2)[:,None,None]*(meanVelGrad-Lij)

In [ ]:
plt.figure()
i1=0
j1=0
#plt.plot(predict[0:576,i1,j1]*hof[0].a[i1,j1]+hof[0].b[i1,j1])
plt.plot(predict[0:576,i1,j1])

#plt.plot(test[0:576,0,0]*hof[0].a[0,0]+hof[0].b[0,0])

plt.plot(Gij[0:576,i1,j1])


In [ ]:
Gij, Tij, meanVelGrad, b, Sij, omega, Lij, eps, K, basis_vect, loindices = get_training_data('../GLM/100/')

In [ ]:
Gij2, Tij2, meanVelGrad2, b2, Sij2, omega2, Lij2, eps2, K2, basis_vect2, loindices2 = get_training_data('../GLM/50/')

In [ ]:
loindices2

In [ ]:
pickle.dump(reg, open('lin_test.pkl', 'wb'))

In [ ]:
eps2.shape

In [ ]:
plt.figure()
plt.plot(np.trace(Gij,axis1=0,axis2=1)*eps/K)
#plt.plot(meanVelGrad2[:,0,0]+meanVelGrad2[:,1,1]+meanVelGrad2[:,2,2])

In [ ]:
basis_vect=get_basis_vect(b,Sij,omega,meanVelGrad,Reyn,Tij)
basis_vect2=get_basis_vect(b2,Sij2,omega2,meanVelGrad2,Reyn2,Tij2)

basis_vect = np.moveaxis(basis_vect,0,-2)
Gij = np.moveaxis(Gij,0,-1)
Tij = np.moveaxis(Tij,0,-1)
Lij = np.moveaxis(Lij,0,-1)
b = np.moveaxis(b,0,-1)
meanVelGrad = np.moveaxis(meanVelGrad,0,-1)


basis_vect2 = np.moveaxis(basis_vect2,0,-2)
Gij2 = np.moveaxis(Gij2,0,-1)
Tij2 = np.moveaxis(Tij2,0,-1)
Lij2 = np.moveaxis(Lij2,0,-1)
b2 = np.moveaxis(b2,0,-1)
meanVelGrad2 = np.moveaxis(meanVelGrad2,0,-1)

In [ ]:
loindices2

In [ ]:
reg = pickle.load(open('lin_test.pkl', 'rb'))

In [ ]:
X2 = np.copy(basis_vect2[:,:,:,0:nfs])
Y2 = np.copy(Gij2)
data2 = np.reshape(X2,[3*3*K2.shape[0],nfs])    
targ2 = np.reshape(Y2,[3*3*K2.shape[0]])
score = np.zeros(len(loindices2)-2)
for i in range(0,len(loindices2)-2):
    pred = reg.predict(data2[loindices2[i]:loindices2[i+1],:])
    score[i] = r2_score(targ2[loindices2[i]:loindices2[i+1]],pred)


In [ ]:
targ2.shape

In [ ]:
pred = reg.predict(data2)
pred = np.reshape(pred,[3,3,K2.shape[0]]) 

In [ ]:
r2_score(targ2[loindices2[0]:loindices[1]],pred[loindices2[0]:loindices[1]])

In [ ]:
plt.figure()
#plt.plot(score,'-o')
plt.plot(pred[0,0,:])
plt.plot(Gij2[0,0,:])


In [ ]:
#nfs=18
X = np.copy(basis_vect[:,:,:,0:nfs])
Y = np.copy(Gij)
data = np.reshape(X,[3*3*K.shape[0],nfs])    
targ = np.reshape(Y,[3*3*K.shape[0]])

#weights = np.ones((3,3,Y.shape[2]))*eps[None,None,:]/K[None,None,:]
#weights = np.reshape(weights,[3*3*K.shape[0]])
    
#weights2 = np.ones((3,3,Y2.shape[2]))*eps2[None,None,:]/K2[None,None,:]
#weights2 = np.reshape(weights2,[3*3*K2.shape[0]])


X_train, X_test, Y_train, Y_test  =  train_test_split(data,targ,test_size=0.2,random_state=1)

mses = np.zeros(40)
for nfs in range(39,40):
    print(nfs)
    slope = np.zeros((3,3,nfs))
    intercept = np.zeros((3,3,nfs))       

      
    
    #reg = LR(normalize=False,fit_intercept=True).fit(data,targ,sample_weight=weights)
    #reg = RandomForestRegressor(n_estimators=100,max_depth=10,max_features=0.8).fit(data,targ)
    #reg = SVR(kernel='linear').fit(data,targ,sample_weight=weights)
    
    #reg = Lasso(alpha=0.001,max_iter=1e6,fit_intercept=True,random_state=10).fit(data,targ)
    #reg = LassoCV(eps=1e-2,fit_intercept=True,cv=8,random_state=10).fit(data,targ)

    #reg = RidgeCV(alphas=(0.5,0.01,0.001,0.0001,0.00005),cv=5).fit(data,targ,sample_weight=weights)
    #data_dmatrix = xgb.DMatrix(data=data,label=targ)
    reg = xgb.XGBRegressor(objective ='reg:squarederror')
    reg.fit(X_train,Y_train)
    



    #mse_t=mean_squared_error(targ,reg.predict(data))
    #mse_v=mean_squared_error(targ2,reg.predict(data2))
    #mses[nfs]=mse_t
    predict = reg.predict(X_test)
    sc = reg.score(X_test,Y_test)
    sc2 = reg.score(X_train,Y_train)

    #predict = np.reshape(predict,[3,3,int(Y_test.shape[0]/9)])

In [ ]:
sc

In [ ]:
plt.figure()
plt.scatter(Y_test,predict,marker='.')

In [ ]:
sc

In [ ]:
predict = np.reshape(predict,[K2.shape[0],3,3])

In [ ]:
plt.figure()
plt.plot(Y2[0,0,:]*eps2/K2)
plt.plot(predict[0,0,:])

In [ ]:
reg.coef_

In [ ]:
imps =[0]
for i in range(1,22):
    imps.append(mses[i-1]-mses[i])
imps[1]=0

In [ ]:
[i for i in imps]


In [ ]:
plt.figure()
plt.bar(range(0,17),imps)

In [ ]:
#nfs=18
mses = np.zeros(37)
for nfs in range(1,37):
    print(nfs)
    slope = np.zeros((3,3,nfs))
    intercept = np.zeros((3,3,nfs))       

      
    X = np.zeros((3,K.shape[0],nfs))
    X2 = np.zeros((3,K2.shape[0],nfs))
    Y = np.zeros((3,K.shape[0]))
    Y2 = np.zeros((3,K2.shape[0]))
    
    for i in range(0,3):
        X[i,:,:] = basis_vect[i,i,:,0:nfs]
        X2[i,:,:] = basis_vect2[i,i,:,0:nfs]
        Y[i,:] = Gij[i,i,:]
        Y2[i,:] = Gij2[i,i,:]
        
        
    
    
    #X = np.copy(basis_vect[0:+1,0:2,:,0:nfs])
    #Y = np.copy(Gij[0:1,0:2,:])
    #X2 = np.copy(basis_vect2[0:1,0:2,:,0:nfs])
    #Y2 = np.copy(Gij2[0:1,0:2,:])
    
    

    data = np.reshape(X,[3*K.shape[0],nfs])
    data2 = np.reshape(X2,[3*K2.shape[0],nfs])
    
    targ = np.reshape(Y,[3*K.shape[0]])
    targ2 = np.reshape(Y2,[3*K2.shape[0]])

    weights = np.ones((1,3,K.shape[0]))*eps[None,None,:]/K[None,None,:]
    weights = np.reshape(weights,[3*K.shape[0]])
    
    weights2 = np.ones((1,3,K2.shape[0]))*eps2[None,None,:]/K2[None,None,:]
    weights2 = np.reshape(weights2,[3*K2.shape[0]])
     
    #reg = LR(normalize=False,fit_intercept=True).fit(data,targ,sample_weight=weights)
    
    #reg = Lasso(alpha=0.0001,max_iter=1e6,fit_intercept=True,random_state=10).fit(data,targ)
    #reg = LassoCV(eps=1e-4,max_iter=1e6,fit_intercept=True,cv=8,random_state=10).fit(data,targ)

    reg = RidgeCV(alphas=(0.01,0.001,0.0001,0.00005,0.00001,1e-6,5e-6,1e-7),cv=5).fit(data,targ,sample_weight=weights)

    mse_t=mean_squared_error(targ,reg.predict(data))
    mse_v=mean_squared_error(targ2,reg.predict(data2))
    mses[nfs-1]=mse_t
    predict = reg.predict(data2)
    sc = reg.score(data2,targ2,sample_weight=weights2)
    predict = np.reshape(predict,[1,3,K2.shape[0]])

In [ ]:
sc

In [ ]:
mses

In [ ]:
reg.alpha_

In [ ]:
imps =[0]
for i in range(1,34):
    imps.append(mses[i-1]-mses[i])
#imps[1]=0

In [ ]:
predict.shape

In [ ]:
plt.figure()
plt.plot(Gij2[2,2,:])
plt.plot(predict[0,2,:])


In [ ]:
predict.shape

In [ ]:
sc

In [ ]:
reg.coef_

In [ ]:
slope[0,0,:]

In [ ]:
mse_t

In [ ]:
nfs = 10
slope = np.zeros((3,3,nfs))
predict = np.zeros((3,3,K2.shape[0]))
sc = np.zeros((3,3))
mse_t=0
mse_v=0

#X = basis_vect[:,:,:,0:nfs]*eps[None,None,:,None]/K[None,None,:,None]
#Y = Gij*eps[None,None,:]/K[None,None,:]

X = np.copy(basis_vect[:,:,:,0:nfs])
Y = np.copy(Gij)

#X2 = basis_vect2[:,:,:,0:nfs]*eps2[None,None,:,None]/K2[None,None,:,None]
#Y2 = Gij2*eps2[None,None,:]/K2[None,None,:]

X2 = np.copy(basis_vect2[:,:,:,0:nfs])
Y2 = np.copy(Gij2)

r2sc = np.zeros((3,3))

for i in range(0,3):
    for j in range(0,3):
        
        
        #reg = LR(fit_intercept=False).fit(X[i,j,:,0:nfs]*eps[:,None]/K[:,None],Y[i,j,:]*eps/K)
        #reg = LassoCV(eps=1e-6,max_iter=1000000,cv=4).fit(basis_vect[i,j,:,0:nfs],Gij[i,j,:])
        
        #reg = RidgeCV(alphas=(0.1,0.01,0.001,0.0001,0.00005,0.00001,1e-6),cv=10).fit(X[i,j,:,0:nfs],Y[i,j,:])
        slope[i,j,:]=reg.coef_

        mse_t = mse_t + mean_squared_error(Y[i,j,:],reg.predict(X[i,j,:,0:nfs]))
        
        predict[i,j,:] = reg.predict(X2[i,j,:,0:nfs]*eps2[:,None]/K2[:,None])
        r2sc[i,j] = r2_score(Y2[i,j,:]*eps2/K2,predict[i,j,:]*eps2/K2)
        mse_v = mse_v + mean_squared_error(Y2[i,j,:],reg.predict(X2[i,j,:,0:nfs]))
        sc[i,j] = reg.score(X2[i,j,:,0:nfs],Y2[i,j,:],sample_weight=eps2/K2)

In [ ]:
slope[:,:,7]

In [ ]:
r2sc

In [ ]:
t = np.reshape(Gij2,[3*3*K2.shape[0]])

In [ ]:
p1 = np.reshape(predict,[3*3*K2.shape[0]])

In [ ]:
r2_score(t,p1)

In [ ]:
predict.shape

In [ ]:
plt.figure(figsize=(10,10))
for i in range(0,3):
    for j in range(0,3):
        plt.subplot(3,3,i*3+j+1) 
        
        #plt.plot(Gij2[i,j,:]*eps2/K2)        
        #plt.plot(predict[i,j,:]*eps2/K2)
        
        #plt.plot(Gij[i,j,:]*eps/K)        
        #plt.plot(predict[i,j,:]*eps/K)
        
        
        #plt.scatter(Gij[i,j,:]*eps/K,predict[i,j,:]*eps/K,marker='.')
        Y2 = np.reshape(Y_test,[3,3,int(Y_test.shape[0]/9)])
        #plt.plot(Y2[i,j,:])
        #plt.plot(predict[i,j,:])
        
        plt.scatter(Y2[i,j,:],predict[i,j,:],marker='.')        
        #plt.plot(Gij[i,j,:]*eps/K)
        #plt.plot(LIPM[i,j,:])
        
        plt.locator_params(axis='y', nbins=4)
        plt.locator_params(axis='x', nbins=3)
        plt.title(str(i+1)+','+str(j+1))
        
        plt.gca().get_yaxis().set_major_formatter(ticker.FormatStrFormatter('%0.0e'))
        plt.gca().get_xaxis().set_major_formatter(ticker.FormatStrFormatter('%0.0e'))
        plt.grid(alpha=0.2)
        #if(i==0 and j==0):
        #    plt.legend(('DNS','Model'))
        plt.subplots_adjust(wspace=0.6)
        plt.subplots_adjust(hspace=0.6)
        #plt.gca().axes.get_yaxis().set_visible(False)
        #plt.gca().axes.get_xaxis().set_visible(False)
        #x = np.linspace(min(Gij[i,j,:]*eps/K),max(Gij[i,j,:]*eps/K),10)        
        
        #plt.plot(x,x,'k--')
#plt.savefig('Gij_linear_reg.png',dpi=300,bbox_inches = "tight")        

In [ ]:
Gijpred = predict*eps2[None,None,:]/K2[None,None,:]

In [ ]:
LIPM2.shape

In [ ]:
RSM = np.zeros((3,3,K2.shape[0]))
RSLIPM = np.zeros((3,3,K2.shape[0]))

for i in range(0,K2.shape[0]):
    RSM[:,:,i]  = np.matmul(Gijpred[:,:,i],Tij2[:,:,i]*K2[i]) + np.transpose(np.matmul(Gijpred[:,:,i],Tij2[:,:,i]*K2[i]))
    RSLIPM[:,:,i]  = np.matmul(LIPM2[:,:,i],Tij2[:,:,i]*K2[i]) + np.transpose(np.matmul(LIPM2[:,:,i],Tij2[:,:,i]*K2[i]))

In [ ]:
plt.figure()
loend=211
hiend=365
plt.plot(RSM[2,1,:])
plt.plot(RSLIPM[2,1,:])
plt.plot(ps[loend:hiend,7]-pt[loend:hiend,7]-epsij[loend:hiend,7])


In [ ]:
ps = np.loadtxt('../GLM/50/press_strain_219.dat')
pt = np.loadtxt('../GLM/50/press_trans_219.dat')
epsij = np.loadtxt('../GLM/50/epsij_219.dat')




In [ ]:
loend

In [ ]:
def LIPM_model(C0,CIPM,alpha2,alpha3,beta1,beta2,beta3,gama5,gama6,b,Tij,meanVelGrad,eps,K,lambdaij):
    pred = np.zeros((3,3,K.shape[0]))
    for i in range(0,K.shape[0]):        
        P = - 0.5*np.trace(np.matmul(Tij[:,:,i],np.transpose(meanVelGrad[:,:,i])) + np.transpose(np.matmul(Tij[:,:,i],np.transpose(meanVelGrad[:,:,i]))))
        B3 = np.matmul(np.matmul(b[:,:,i],b[:,:,i]),b[:,:,i])
        
        alpha1 = -(0.5+3.0/4*C0)+3.0*alpha2*np.trace(B3)+0.5*CIPM*P #P is non dim so no omega here
        
        pred[:,:,i] = alpha1*np.eye(3,3)+alpha2*b[:,:,i] + alpha3*np.matmul(b[:,:,i],b[:,:,i]) + beta1*np.eye(3,3)*np.trace(meanVelGrad[:,:,i]) + \
        beta2*meanVelGrad[:,:,i] + beta3*np.transpose(meanVelGrad[:,:,i]) + gama5*np.matmul(b[:,:,i],meanVelGrad[:,:,i]) + \
        gama6*np.matmul(b[:,:,i],np.transpose(meanVelGrad[:,:,i]))
        pred[:,:,i] = pred[:,:,i]*eps[i]/K[i]
        pred[:,:,i] = pred[:,:,i]+0.5*C0*eps[i]/K[i]*lambdaij[:,:,i]
    return pred

    

In [ ]:
Lij.shape

In [ ]:
LIPM  = LIPM_model(2.1,0.6,3.5,-3*3.5,-0.2,0.8,-0.2,0.6,-0.6,b,Tij,meanVelGrad,eps,K,Lij)
LIPM2 = LIPM_model(2.1,0.6,3.5,-3*3.5,-0.2,0.8,-0.2,0.6,-0.6,b2,Tij2,meanVelGrad2,eps2,K2,Lij2)

In [ ]:
def get_basis_LIPM(C0,CIPM,alpha2,alpha3,beta1,beta2,beta3,gama5,gama6,b,Tij,meanVelGrad):
        basis_vect = np.zeros((b.shape[2],3,3,10))
        for i in range(0,b.shape[2]):
            basis_vect[i,:,:,0]=np.eye(3,3)
            basis_vect[i,:,:,1]=b[:,:,i]
            basis_vect[i,:,:,2]=np.matmul(b[:,:,i],b[:,:,i])
            basis_vect[i,:,:,3]=np.trace(meanVelGrad[:,:,i])*np.eye(3,3)        
            basis_vect[i,:,:,4]=meanVelGrad[:,:,i]
            basis_vect[i,:,:,5]=np.transpose(meanVelGrad[:,:,i])
            basis_vect[i,:,:,6]=np.matmul(b[:,:,i],meanVelGrad[:,:,i])
            basis_vect[i,:,:,7]=np.matmul(b[:,:,i],np.transpose(meanVelGrad[:,:,i]))
            basis_vect[i,:,:,8]=np.trace(np.matmul(np.matmul(b[:,:,i],b[:,:,i]),b[:,:,i]))*np.eye(3,3)
            basis_vect[i,:,:,9]=- 0.5*np.trace(np.matmul(Tij[:,:,i],np.transpose(meanVelGrad[:,:,i])) +\
                                np.transpose(np.matmul(Tij[:,:,i],np.transpose(meanVelGrad[:,:,i]))))*np.eye(3,3)
        basis_vect = np.moveaxis(basis_vect,0,-2)
        return basis_vect
            

    

In [ ]:
b.shape

In [ ]:
basis_LIPM = get_basis_LIPM(2.1,0.6,3.5,-3*3.5,-0.2,0.8,-0.2,0.6,-0.6,b,Tij,meanVelGrad)
basis_LIPM2 = get_basis_LIPM(2.1,0.6,3.5,-3*3.5,-0.2,0.8,-0.2,0.6,-0.6,b2,Tij2,meanVelGrad2)


In [ ]:
P= -0.5*np.trace(np.matmul(Tij[:,:,i],np.transpose(meanVelGrad[:,:,i])) +\
                                np.transpose(np.matmul(Tij[:,:,i],np.transpose(meanVelGrad[:,:,i]))))

In [ ]:
P